In [3]:
from DataProcess.utils.filehelper import read_csv_file
from tqdm import tqdm
import csv

goods_same = read_csv_file('./data/goods_same.csv')[1:]
goods_similar = read_csv_file('./data/goods_similar_correct.csv')[1:]

1233383it [00:03, 396392.31it/s]
104171it [00:00, 328142.61it/s]


In [9]:
from DataProcess.utils.filehelper import join_prompt_strings
# 有422个长度超过256

final_data = []
max_length = 0
num_max_length = 0
now_str = ''
id = 0

for data in tqdm(goods_similar):
    if len(data)==7:
        if data[5] == 1:
            pass
        else:
            now_str = join_prompt_strings(prompt="对比相同和相似物资:",str_a = data[3],str_b = data[4])
            if len(now_str)>256:
                num_max_length+=1
            if len(now_str)>max_length:
                max_length = len(now_str)
            else:
                final_data.append([id, now_str,data[5],data[6]])
                id += 1

for data in tqdm(goods_same):
    if len(data) == 4:
        now_str = join_prompt_strings(prompt="对比相同和相似物资:",str_a = data[0],str_b = data[1])
        if len(now_str)>256:
                num_max_length+=1
        if len(now_str)>max_length:
            max_length = len(now_str)
        else:
            final_data.append([id,now_str,data[2],data[3]])
            id += 1
print(num_max_length)

100%|██████████| 1233382/1233382 [00:04<00:00, 265237.95it/s]

422


In [10]:
from sklearn.model_selection import train_test_split

def split_data(data_list, test_size=0.1,stratify=False, random_state=42):
    X = [data[0:2] for data in data_list]
    y = [data[2:] for data in data_list]
    if stratify:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y)
    else:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    train_list = [x + list(y) for x, y in zip(X_train, y_train)]
    test_list = [x + list(y) for x, y in zip(X_test, y_test)]
    return train_list, test_list

def export_to_csv(data_list, csv_path):
    with open(csv_path, 'w', encoding='utf-8', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['id','data', 'same', 'similar'])  # 表头
        for i, data in enumerate(data_list):
            writer.writerow(data)

train_data, test_data  =  split_data(final_data, test_size=0.1,stratify=True, random_state=114514)

In [11]:
export_to_csv(train_data,'./processed/train.csv')
export_to_csv(test_data, './processed/test.csv')